model로드시 pretraine 때 사용했던 토크나이저를 그대로 사용해야 벡터 변환에 문제가 없음.

In [1]:
user_defined_symbols = ['[PAD]', '[UNK]', '[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]', '[CLS]', '[SEP]', '[MASK]', '[BOS]','[EOS]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

In [2]:
# load tokenizer
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast(
    vocab_file = './hf_tokenizer_special/vocab.txt',
    max_len = 128,
    do_lower_case=False,
)

special_token_dic = {'additional_special_tokens': user_defined_symbols}
tokenizer.add_special_tokens(special_token_dic)
print(tokenizer.tokenize('曔은 [MASK] 엄청 맛있었고 촉촉하고 바삭했어요.'))

['[UNK]', '은', '[MASK]', '엄청', '맛있', '##었', '##고', '촉촉', '##하고', '바삭', '##했', '##어요', '.']


### Input Formatting
- special token[sep]은 문장의 끝을 표시하거나 두 문장의 분리할 때 사용한다.
- special token[CLS]은 문장 시작할 때 사용한다. 이 토큰은 분류 문제에 사용되지만, 어떤 문제를 풀더라도 입력해야한다.
- BERT에서 사용되는 단어사전에 있는 토큰
- BERT 토크 나이저의 토큰에 대한 Token ID
- 시퀀스에서 어떤 요소가 토큰이고 패딩 요소인지를 나타내는 Mask ID
- 다른 문장을 구별하는데 사용되는 Segment ID
- 시퀀스 내에서 토큰 위치를 표시하는 데 사용되는 Positional Embeddings <br>
tokenizer
tokenize.encode_plus

In [3]:
test_txt = "[CLS]오늘 서촌에 갔다가 복숭아 요거트를 먹었어요. 우연히 들어간 곳이었는데 정말 맛있었답니다.[SEP]"
tokened_txt = tokenizer.tokenize(test_txt)
indexed_txt = tokenizer.convert_tokens_to_ids(tokened_txt)
for t, i in zip(tokened_txt, indexed_txt):
    print(f'token: {t}, index: {i}')

token: [CLS], index: 2
token: 오늘, index: 6520
token: 서촌, index: 26906
token: ##에, index: 4444
token: 갔다, index: 9930
token: ##가, index: 4391
token: 복숭아, index: 10138
token: 요거, index: 7991
token: ##트, index: 4189
token: ##를, index: 4633
token: 먹, index: 1441
token: ##었, index: 4229
token: ##어요, index: 6682
token: ., index: 9
token: 우연히, index: 9084
token: 들어간, index: 7153
token: 곳, index: 195
token: ##이, index: 4196
token: ##었, index: 4229
token: ##는데, index: 9223
token: 정말, index: 6490
token: 맛있, index: 6474
token: ##었, index: 4229
token: ##답니다, index: 7219
token: ., index: 9
token: [SEP], index: 3


In [6]:
segements_ids = []
seg_num = 1
for token in tokened_txt:
    segements_ids.append(seg_num)
    if token == "[SEP]":
        seg_num -= 1
print(segements_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


bert pytorch interface에서는 데이터 형태가 python list(X)<br>
데이터를 torch tensor로 변환하고 BERT model call. 

In [8]:
import torch

# convert inputs to torch tensor
token_tensor = torch.tensor([indexed_txt])
segments_tensors = torch.tensor([segements_ids])

In [111]:
# model load
from transformers import BertConfig, BertModel

pretrained_model_config = BertConfig.from_pretrained('model_output')

my_model = BertModel.from_pretrained(
    'model_output', 
    # output_hidden_states=True,
    config=pretrained_model_config
)
pretrained_model_config

Some weights of the model checkpoint at model_output were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertConfig {
  "_name_or_path": "model_output",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embedding": 128,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [19]:
# model.eval()은 평가 모드로 모델 선정, 드롭아웃 해제
# torch.no_grad()는 forward만, backprop하지 X 메모리랑 속도 
with torch.no_grad():
    outputs = my_model(token_tensor, segments_tensors)
    hidden_states = outputs[2]

- outputs[2]
    - 문장이 각각 768 차원 벡터로 변환된 것.
    - Bert 마지막 레이어의 CLS 벡터들
    - 문장 전체를 벡터 하나로 변환한 것

### outputs
- 모델의 여러 출력 결과를 한데 묶은 것
- hidden_states 개체의 저장된 모델의 전체 은닉층은 4개의 차원이 존재
1. The **layer** number (13 layers) bert 12 layers + 1 input embedding
2. The **batch** number (1 sentence)
3. The word / **token** number (36 tokens in our sentence)
4. The hidden unit / **feature** number (768 features)

In [23]:
# 현재 차원 [layers, batches, tokens, features]
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 26
Number of hidden units: 768


In [27]:
# 차원을 [tokens, layers, features]로 만든다. permute 함수 이용
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 26, 768])


In [28]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size() # [layers, batches, tokens, features]

torch.Size([13, 1, 26, 768])

In [30]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 26, 768])

In [31]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size() # [tokens, layers, features]

torch.Size([26, 13, 768])

need to extract each vector of tokens or whole vector of sentence <br>
but we have hidden states and 768 size 13 vector per each 26 tokens(26개 토큰에 대해 768크기의 13개 개별 벡터) <br>
개별 벡터를 얻으려면 일부 레이어 벡터를 결합해야 한다. <br>
결합에 있어 레이어 조합에 관한 최상의 표현 방법은 정해지지 않았음. <br>

In [33]:
# Token vector
# 1. 마지막 4개의 레이어를 concatenate하여 토큰 당 단일 단어 벡터를 제공.
# 각 벡터의 길이 4*768 = 3,072
token_vecs_cat = []
for token in token_embeddings:
    # 1 token -> [12*768], [26*12*768]
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    token_vecs_cat.append(cat_vec)

print('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 26 x 3072


In [35]:
# 2. 마지막 4개의 레이어를 합산(sum)하여 단어 벡터를 만듬.
token_vecs_sum = []
for token in token_embeddings:
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
    
print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 26 x 768


In [41]:
# Sentence Vector
# 1. 토큰별 768크기의 벡터를 생성하는 각 토큰의 두번째에서 마지막 숨겨진 레이어의 평균을 내는 것
# hidden_states -> [13x1x26x768]
token_vecs = hidden_states[-2][0]
# the avg of all 26 token vectors
sentence_embedding = torch.mean(token_vecs, dim=0)
sentence_embedding[:2]

tensor([ 0.2208, -0.5151])

In [47]:
sentence_embedding.size()

torch.Size([768])

### word embed
1. last_hidden_state = outputs\[0\]
    - word_embed_1 = last_hidden_state
2. initial embeddings can be taken from 0th layer of hidden states
    - word_embed_2 = hidden_states\[0\]
3. sum of all hidden states
    - word_embed_3 = torch.stack(hidden_states).sum(0)
4. sum of second to last layer
    - word_embed_4 = torch.stack(hidden_states\[2:\]).sum(0)
5. sum of last four layer
    - word_embed_5 = torch.stack(hidden_states\[-4:\]).sum(0)
6. concatenate last four layers
    - word_embed_6 = torch.cat(\[hidden_states[i\] for i in \[-1,-2,-3,-4\]\], dim=-1)

In [122]:
def get_token_vec_sum(model, sentence):
    inputs = tokenizer(sentence, return_tensors='pt') # input_ids, token_type_ids(segement info), attention_mask(padding token)
    model.eval()
    outputs = model(**inputs)
    
    hidden_states = outputs.last_hidden_state # or outpus[0]
    token_embeddings = hidden_states.permute(1,0,2)
    
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    # 마지막 4개 레이어 합산한 벡터값
    return inputs, token_vecs_sum # size : (token 갯수, 768 features)

In [139]:
# 상황별 토큰 벡터 비교
t1 = "밤 몽블랑 먹었는데 너무 맛있네요."
t2 = "바밤바보다 맛있는 왕밤빵."
t1_tokens = tokenizer.tokenize("[CLS]" + t1 + "[SEP]")
t2_tokens = tokenizer.tokenize("[CLS]" + t2 + "[SEP]")

input_t1, token_vec_sum_t1 = get_token_vec_sum(my_model, t1)
input_t2, token_vec_sum_t2 = get_token_vec_sum(my_model, t2)
print(len(token_vec_sum_t1), len(token_vec_sum_t1[0])) # (token갯수+[CLS][SEP]토큰2개, 768 features)
print(len(token_vec_sum_t2), len(token_vec_sum_t2[0]))

for i, token_id in enumerate(zip(t1_tokens, input_t1['input_ids'][0])):
    print(i, token_id)
print()
for i, token_id in enumerate(zip(t2_tokens, input_t2['input_ids'][0])):
    print(i, token_id)

11 768
12 768
0 ('[CLS]', tensor(2))
1 ('밤', tensor(1624))
2 ('몽블랑', tensor(18026))
3 ('먹', tensor(1441))
4 ('##었', tensor(4229))
5 ('##는데', tensor(9223))
6 ('너무', tensor(6479))
7 ('맛있', tensor(6474))
8 ('##네요', tensor(18605))
9 ('.', tensor(9))
10 ('[SEP]', tensor(3))

0 ('[CLS]', tensor(2))
1 ('바', tensor(1611))
2 ('##밤', tensor(5030))
3 ('##바', tensor(4298))
4 ('##보다', tensor(29998))
5 ('맛있', tensor(6474))
6 ('##는', tensor(4214))
7 ('왕', tensor(2594))
8 ('##밤', tensor(5030))
9 ('##빵', tensor(4738))
10 ('.', tensor(9))
11 ('[SEP]', tensor(3))


In [132]:
print(f"{t1} 의 '밤'(#1)\n-> {token_vec_sum_t1[1][:5]}") # 768 중 5개까지만 확인
print(f"{t2} 의 '밤'(#8)\n-> {token_vec_sum_t2[5][:5]}")

밤으로 만든 몽블랑 먹었는데 너무 맛있네요. 의 '밤'(#1)
-> tensor([ 1.0066, -1.2224,  0.2787, -0.1907, -0.0580], grad_fn=<SliceBackward>)
바밤바가 너무 맛있어 의 '밤'(#8)
-> tensor([-0.5164, -1.8232,  1.4110, -1.2231, -2.3583], grad_fn=<SliceBackward>)


In [140]:
from scipy.spatial.distance import cosine

# Consine similarity 비교
sim_word = cosine(token_vec_sum_t1[1].detach().numpy(), token_vec_sum_t2[8].detach().numpy())
sim_word

0.5491033792495728